In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import json
import numpy as np
from tqdm import tqdm
from PIL import Image

In [ ]:
data_path = "/content/drive/MyDrive/Computer Vision Final Project/"
os.listdir(data_path)

In [ ]:
with open(data_path+r"/sample.json") as f:
    data = json.load(f)["0"]
    print("Type:", type(data))
 

In [ ]:
X_data_paths = []
y_data = []

for sample in tqdm(data):
    panoid = sample["panoid"]
    image_path = data_path + "training_images_full_pano/full_pano_" + panoid + ".png" 
    if os.path.exists(image_path):
        X_data_paths.append(image_path)
        y_data.append([sample["lng"],sample["lat"]])
    else: 
        pass

X_data_paths = np.array(X_data_paths)
y_data = np.array(y_data)

100%|██████████| 9278/9278 [00:50<00:00, 185.50it/s] 


In [ ]:
assert(len(y_data) == len(X_data_paths))
print(len(X_data_paths))

9274


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_data_paths, y_data, test_size=0.1, random_state=1)

X_train, X_valid, y_train, y_val = train_test_split(X_train, y_train, test_size=0.05, random_state=1) 

In [ ]:
######### TEMP ###########

In [ ]:
from tf.keras.preprocessing.image import img_to_array,load_img

# Define image size and file paths
image_size = (320, 640, 3)

# Create an empty NumPy array to hold the images
X_train_temp = np.zeros((len(X_train), *image_size), dtype=np.float16)

# Load the images into the NumPy array
for i, filename in tqdm(enumerate(X_train)):
    img = load_img(filename, target_size=image_size, color_mode="rgb")
    img_array = img_to_array(img) / 255.0
    X_train_temp[i] = img_array

# Save the NumPy array to a binary file
np.save(data_path+"X_train_90.npy", X_train_temp)

# Load the NumPy array from the binary file
#X_train = np.load(data_path+"X_train.npy")

X_train = X_train_temp

7928it [48:10,  2.74it/s]


In [ ]:
X_val_temp = np.zeros((len(X_valid), *image_size), dtype=np.float16)

# Load the images into the NumPy array
for i, filename in tqdm(enumerate(X_valid)):
    img = load_img(filename, target_size=image_size, color_mode="rgb")
    img_array = img_to_array(img) / 255.0
    X_val_temp[i] = img_array

# Save the NumPy array to a binary file
np.save(data_path+"X_valid_90_95.npy", X_val_temp)

418it [02:37,  2.65it/s]


In [ ]:
X_train = np.load(data_path+"X_train_90.npy")
X_valid = np.load(data_path+"X_valid_90_95.npy")

In [ ]:
#X_training = [] 

#for x in tqdm(X_train[0:100]):
#    X_training.append(np.asarray(Image.open(x)))
#X_train = np.array(X_training)



In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Conv2D, Conv2DTranspose, MaxPool2D, Cropping2D, BatchNormalization, Activation, Dropout, Flatten
from tensorflow.keras import Model, Input
import numpy as np
from tensorflow.keras.preprocessing.image import load_img
import random
import os


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13783825667559558244
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14343274496
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13142232546706530379
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [ ]:
class ImageData(tf.keras.utils.Sequence):
    def __init__(self, batch_size, img_size, input_img_paths, target_list):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_img_paths = np.array(input_img_paths)
        self.targets = np.array(target_list)

    def __len__(self):
        return len(self.targets) // self.batch_size

    def __getitem__(self, idx):

        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_target_list = self.targets[i : i + self.batch_size]
        x = np.zeros((self.batch_size,) + self.img_size , dtype="float32")

        for j, path in enumerate(batch_input_img_paths):
            img = load_img(path, target_size=self.img_size)
            x[j] = img
        y = np.zeros((self.batch_size,)+(2,), dtype="float32")
        for j, target in enumerate(batch_target_list):
            y[j] = target
        return x, y

In [ ]:
class ImageData2(tf.keras.utils.Sequence):
    def __init__(self, batch_size, img_size, input_imgs, target_list):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_imgs = input_imgs
        self.targets = target_list

    def __len__(self):
        return len(self.targets) // self.batch_size

    def __getitem__(self, idx):
        i = idx * self.batch_size
        x = self.input_imgs[i : i + self.batch_size]
        y = self.targets[i : i + self.batch_size]
        return x, y

In [ ]:
def InceptionBlock(prev_layer, first_filters, second_filters, thrid_filter, fourth_filter, dropout_rate=0.0):
    first_layer = Conv2D(first_filters, 1, padding="same", kernel_initializer="he_normal", bias_initializer="zeros")(prev_layer)
    first_layer = BatchNormalization()(first_layer)
    first_layer = Activation("relu")(first_layer)
    second_layer = Conv2D(second_filters[0], 1, padding="same", kernel_initializer="he_normal", bias_initializer="zeros")(prev_layer)
    second_layer = BatchNormalization()(second_layer)
    second_layer = Activation("relu")(second_layer)
    second_layer = Conv2D(second_filters[1], 3, padding="same", kernel_initializer="he_normal", bias_initializer="zeros")(second_layer)
    second_layer = BatchNormalization()(second_layer)
    second_layer = Activation("relu")(second_layer)
    third_layer = Conv2D(thrid_filter[0], 1, padding="same", kernel_initializer="he_normal", bias_initializer="zeros")(prev_layer)
    third_layer = BatchNormalization()(third_layer)
    third_layer = Activation("relu")(third_layer)
    third_layer = Conv2D(thrid_filter[1], 5, padding="same", kernel_initializer="he_normal", bias_initializer="zeros")(third_layer)
    third_layer = BatchNormalization()(third_layer)
    third_layer = Activation("relu")(third_layer)
    fourth_layer = MaxPool2D((3, 3), strides=(1, 1), padding="same")(prev_layer)
    fourth_layer = Conv2D(fourth_filter, 5, padding="same", kernel_initializer="he_normal", bias_initializer="zeros")(fourth_layer)
    fourth_layer = BatchNormalization()(fourth_layer)
    fourth_layer = Activation("relu")(fourth_layer)
    prev_layer = tf.concat((first_layer, second_layer, third_layer, fourth_layer), axis=-1)
    if dropout_rate > 0.0:
        prev_layer = Dropout(dropout_rate)(prev_layer)
    return prev_layer

In [ ]:
def InceptionNetwork(input_size=(320, 640, 3),dropout_rate = 0.1, dropout_rate_end = 0.2):
    # Base module
    inputs = Input(shape = input_size)
    enc1 = Conv2D(filters=32, kernel_size=3, strides=(2,2), padding="same", kernel_initializer="he_normal", bias_initializer="zeros")(inputs)
    enc1 = BatchNormalization()(enc1)
    enc1 = Activation("relu")(enc1)
    if dropout_rate > 0.0:
        enc1 = Dropout(dropout_rate)(enc1)
    enc2 = MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding="same")(enc1)
    enc2 = Conv2D(filters=32, kernel_size=3, padding="same", kernel_initializer="he_normal", bias_initializer="zeros")(enc2)
    enc2 = BatchNormalization()(enc2)
    enc2 = Activation("relu")(enc2)
    enc2 = Conv2D(filters=32, kernel_size=3, padding="same", kernel_initializer="he_normal", bias_initializer="zeros")(enc2)
    enc2 = BatchNormalization()(enc2)
    enc2 = Activation("relu")(enc2)
    # Inception Part
    if dropout_rate > 0.0:
        enc2 = Dropout(dropout_rate)(enc2)
    enc3 = MaxPool2D((3, 3), strides=(2, 2), padding="same")(enc2)
    enc3 = InceptionBlock(enc3, 32, (64, 128), (16, 32), 32, dropout_rate=dropout_rate)
    #enc3 = InceptionBlock(enc3, 64, (128, 192), (32, 64), 64, dropout_rate=dropout_rate)
    #enc3 = InceptionBlock(enc3, 128, (128, 256), (64, 128), 128, dropout_rate=dropout_rate)
    if dropout_rate > 0.0:
        enc3 = Dropout(dropout_rate)(enc3)
    # compress to a square img by using max pool
    enc4 = MaxPool2D((3, 3), strides=(2, 1), padding="same")(enc3)
    enc4 = InceptionBlock(enc4, 32, (64, 128), (16, 32), 32, dropout_rate=dropout_rate)
    #enc4 = InceptionBlock(enc4, 64, (128, 192), (32, 64), 64, dropout_rate=dropout_rate)
    #enc4 = InceptionBlock(enc4, 128, (128, 256), (64, 128), 128, dropout_rate=dropout_rate)
    if dropout_rate > 0.0:
        enc4 = Dropout(dropout_rate)(enc4)
    #enc5 = MaxPool2D((4, 4), strides=(4, 4), padding="same")(enc4)
    #enc5 = InceptionBlock(enc5, 32, (64, 128), (16, 32), 32, dropout_rate=dropout_rate)
    #enc5 = InceptionBlock(enc5, 64, (128, 192), (32, 64), 64, dropout_rate=dropout_rate)
    #enc5 = InceptionBlock(enc5, 128, (128, 256), (64, 128), 128, dropout_rate=dropout_rate)

    # Final layers
    enc6 = MaxPool2D((4, 4), strides=(4, 4), padding="same")(enc4)
    enc6 = Dropout(dropout_rate_end)(enc6)
    enc6 = Flatten()(enc6)
    lastlayer_coordinates = Dense(2,)(enc6)

    #lastlayer_country = Dense(187,activation = "softmax")(enc6)
    #output = tf.concat((lastlayer_coordinates,lastlayer_country), axis=-1)
    output = lastlayer_coordinates
    return Model(inputs=inputs, outputs=output)

In [ ]:
def InceptionNetwork_small(input_size=(320, 640, 3),dropout_rate = 0.1, dropout_rate_end = 0.2):
    # Base module
    inputs = Input(shape = input_size)
    enc1 = Conv2D(filters=8, kernel_size=3, strides=(2,2), padding="same", kernel_initializer="he_normal", bias_initializer="zeros")(inputs)
    enc1 = BatchNormalization()(enc1)
    enc1 = Activation("relu")(enc1)
    if dropout_rate > 0.0:
        enc1 = Dropout(dropout_rate)(enc1)
    enc2 = MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding="same")(enc1)
    enc2 = Conv2D(filters=8, kernel_size=3, padding="same", kernel_initializer="he_normal", bias_initializer="zeros")(enc2)
    enc2 = BatchNormalization()(enc2)
    enc2 = Activation("relu")(enc2)
    enc2 = Conv2D(filters=8, kernel_size=3, padding="same", kernel_initializer="he_normal", bias_initializer="zeros")(enc2)
    enc2 = BatchNormalization()(enc2)
    enc2 = Activation("relu")(enc2)
    # Inception Part
    if dropout_rate > 0.0:
        enc2 = Dropout(dropout_rate)(enc2)
    enc3 = MaxPool2D((3, 3), strides=(2, 2), padding="same")(enc2)
    enc3 = InceptionBlock(enc3, 8, (16, 32), (4, 8), 8, dropout_rate=dropout_rate)
    #enc3 = InceptionBlock(enc3, 64, (128, 192), (32, 64), 64, dropout_rate=dropout_rate)
    #enc3 = InceptionBlock(enc3, 128, (128, 256), (64, 128), 128, dropout_rate=dropout_rate)
    if dropout_rate > 0.0:
        enc3 = Dropout(dropout_rate)(enc3)
    # compress to a square img by using max pool
    enc4 = MaxPool2D((3, 3), strides=(2, 1), padding="same")(enc3)
    enc4 = InceptionBlock(enc4, 8, (16, 32), (4, 8), 8, dropout_rate=dropout_rate)
    #enc4 = InceptionBlock(enc4, 64, (128, 192), (32, 64), 64, dropout_rate=dropout_rate)
    #enc4 = InceptionBlock(enc4, 128, (128, 256), (64, 128), 128, dropout_rate=dropout_rate)
    if dropout_rate > 0.0:
        enc4 = Dropout(dropout_rate)(enc4)
    enc5 = MaxPool2D((4, 4), strides=(2, 2), padding="same")(enc4)
    enc5 = InceptionBlock(enc5, 16, (32, 64), (8, 16), 16, dropout_rate=dropout_rate)
    #enc5 = InceptionBlock(enc5, 64, (128, 192), (32, 64), 64, dropout_rate=dropout_rate)
    #enc5 = InceptionBlock(enc5, 128, (128, 256), (64, 128), 128, dropout_rate=dropout_rate)

    # Final layers
    enc6 = MaxPool2D((4, 4), strides=(4, 4), padding="same")(enc5)
    enc6 = Dropout(dropout_rate_end)(enc6)
    enc6 = Flatten()(enc6)
    lastlayer_coordinates = Dense(2,)(enc6)

    #lastlayer_country = Dense(187,activation = "softmax")(enc6)
    #output = tf.concat((lastlayer_coordinates,lastlayer_country), axis=-1)
    output = lastlayer_coordinates
    return Model(inputs=inputs, outputs=output)

In [ ]:
model= InceptionNetwork()
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 320, 640, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_111 (Conv2D)            (None, 160, 320, 8)  224         ['input_6[0][0]']                
                                                                                                  
 batch_normalization_111 (Batch  (None, 160, 320, 8)  32         ['conv2d_111[0][0]']             
 Normalization)                                                                                   
                                                                                            

In [ ]:
#x_train = np.array([np.asarray(Image.open(x)) for x in X_train_paths])
#y_train = np.array(y_train)

In [ ]:
def normalize_coordinates(coords):
    normalized_coords = np.zeros_like(coords)
    normalized_coords[:, 1] = (coords[:, 1]) / 90
    normalized_coords[:, 0] = (coords[:, 0]) / 180
    return normalized_coords

def inverse_normalize_coordinates(normalized_coords):
    coords = np.zeros_like(normalized_coords)
    coords[:, 1] = (normalized_coords[:, 1]) * 90
    coords[:, 0] = (normalized_coords[:, 0]) * 180
    return coords

In [ ]:
y_train_norm = normalize_coordinates(y_train)
y_valid_norm = normalize_coordinates(y_val)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
lossfunc = 'mean_squared_error'
lossfunc = 'log_cosh'
##
version = "0.14_norm_small_logcosh"

logpath = data_path + r"/model/logs/"
log_dir = logpath +f'inception_network_{version}'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

checkpoint_path = data_path + r"/model/checkpoints/"
checkpoint_dir = checkpoint_path + f'inception_{version}'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_dir, monitor='val_loss', save_best_only=True, mode='min', verbose=0)
##
earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)


In [ ]:
batch_size = 32
image_size = (320,640,3)
training_data = ImageData2(batch_size,image_size,X_train,y_train_norm)

del X_train

In [ ]:
model.compile(
    optimizer=opt,
    loss=lossfunc,
    metrics=['mae','mape']
    )
print('Model compiled')
#with tf.device("/gpu:0"):
history = model.fit(
    training_data,
    epochs=1000,
    batch_size=batch_size,
    verbose=1,
    #validation_split = 0.1,
    validation_data = (X_valid,y_valid_norm),
    callbacks=[tensorboard_callback,checkpoint_callback,earlystop_callback])

Model compiled
Epoch 1/1000
247/247 [==============================] - ETA: 0s - loss: 0.9569 - mae: 1.4315 - mape: 26470.6738

247/247 [==============================] - 60s 199ms/step - loss: 0.9569 - mae: 1.4315 - mape: 26470.6738 - val_loss: 0.1364 - val_mae: 0.4419 - val_mape: 1243.7106
Epoch 2/1000
247/247 [==============================] - ETA: 0s - loss: 0.3137 - mae: 0.7043 - mape: 22915.7246

247/247 [==============================] - 45s 181ms/step - loss: 0.3137 - mae: 0.7043 - mape: 22915.7246 - val_loss: 0.0638 - val_mae: 0.2946 - val_mape: 281.9380
Epoch 3/1000
247/247 [==============================] - ETA: 0s - loss: 0.1565 - mae: 0.4733 - mape: 76814.1250

247/247 [==============================] - 45s 180ms/step - loss: 0.1565 - mae: 0.4733 - mape: 76814.1250 - val_loss: 0.0619 - val_mae: 0.2926 - val_mape: 282.7767
Epoch 4/1000
247/247 [==============================] - ETA: 0s - loss: 0.1022 - mae: 0.3704 - mape: 12769.3525

247/247 [==============================] - 45s 181ms/step - loss: 0.1022 - mae: 0.3704 - mape: 12769.3525 - val_loss: 0.0571 - val_mae: 0.2774 - val_mape: 261.7364
Epoch 5/1000
247/247 [==============================] - 37s 150ms/step - loss: 0.0719 - mae: 0.3077 - mape: 7217.3306 - val_loss: 0.0658 - val_mae: 0.3145 - val_mape: 384.5019
Epoch 6/1000
247/247 [==============================] - 37s 150ms/step - loss: 0.0646 - mae: 0.2887 - mape: 23687.0703 - val_loss: 0.0713 - val_mae: 0.3195 - val_mape: 607.1440
Epoch 7/1000
247/247 [==============================] - 37s 150ms/step - loss: 0.0604 - mae: 0.2767 - mape: 26898.8965 - val_loss: 0.0583 - val_mae: 0.2886 - val_mape: 344.8649
Epoch 8/1000
247/247 [==============================] - 37s 150ms/step - loss: 0.0561 - mae: 0.2646 - mape: 2081.2559 - val_loss: 0.0595 - val_mae: 0.2985 - val_mape: 226.5422
Epoch 9/1000
247/247 [==============================] - ETA: 0s - loss: 0.0553 - mae: 0.2645 - mape: 10368.6934

247/247 [==============================] - 45s 181ms/step - loss: 0.0553 - mae: 0.2645 - mape: 10368.6934 - val_loss: 0.0547 - val_mae: 0.2649 - val_mape: 357.4768
Epoch 10/1000
247/247 [==============================] - ETA: 0s - loss: 0.0503 - mae: 0.2492 - mape: 5325.0425

247/247 [==============================] - 45s 183ms/step - loss: 0.0503 - mae: 0.2492 - mape: 5325.0425 - val_loss: 0.0524 - val_mae: 0.2676 - val_mape: 238.9165
Epoch 11/1000
247/247 [==============================] - 37s 150ms/step - loss: 0.0495 - mae: 0.2471 - mape: 9768.8662 - val_loss: 0.0604 - val_mae: 0.2874 - val_mape: 477.7021
Epoch 12/1000
247/247 [==============================] - 37s 150ms/step - loss: 0.0483 - mae: 0.2432 - mape: 5142.5601 - val_loss: 0.0570 - val_mae: 0.2786 - val_mape: 397.0027
Epoch 13/1000
247/247 [==============================] - 37s 150ms/step - loss: 0.0476 - mae: 0.2412 - mape: 7977.6406 - val_loss: 0.0563 - val_mae: 0.2686 - val_mape: 458.8050
Epoch 14/1000
247/247 [==============================] - 37s 150ms/step - loss: 0.0456 - mae: 0.2366 - mape: 14231.3271 - val_loss: 0.0548 - val_mae: 0.2756 - val_mape: 338.1461
Epoch 15/1000
247/247 [==============================] - ETA: 0s - loss: 0.0444 - mae: 0.2320 - mape: 11573.7900

247/247 [==============================] - 45s 181ms/step - loss: 0.0444 - mae: 0.2320 - mape: 11573.7900 - val_loss: 0.0502 - val_mae: 0.2552 - val_mape: 223.1458
Epoch 16/1000
247/247 [==============================] - 37s 150ms/step - loss: 0.0438 - mae: 0.2313 - mape: 7963.2646 - val_loss: 0.0505 - val_mae: 0.2593 - val_mape: 232.4502
Epoch 17/1000
247/247 [==============================] - 37s 150ms/step - loss: 0.0418 - mae: 0.2247 - mape: 7791.1353 - val_loss: 0.0559 - val_mae: 0.2802 - val_mape: 284.1850
Epoch 18/1000
247/247 [==============================] - 37s 150ms/step - loss: 0.0417 - mae: 0.2260 - mape: 10023.4199 - val_loss: 0.0536 - val_mae: 0.2742 - val_mape: 361.8288
Epoch 19/1000
247/247 [==============================] - 37s 149ms/step - loss: 0.0405 - mae: 0.2212 - mape: 5555.2656 - val_loss: 0.0611 - val_mae: 0.2983 - val_mape: 393.5879
Epoch 20/1000
247/247 [==============================] - 37s 149ms/step - loss: 0.0395 - mae: 0.2185 - mape: 6965.9351 - val_lo

247/247 [==============================] - 45s 182ms/step - loss: 0.0332 - mae: 0.2002 - mape: 14700.1650 - val_loss: 0.0500 - val_mae: 0.2580 - val_mape: 275.4715
Epoch 28/1000
247/247 [==============================] - 37s 151ms/step - loss: 0.0344 - mae: 0.2036 - mape: 33424.8477 - val_loss: 0.0523 - val_mae: 0.2694 - val_mape: 235.6972
Epoch 29/1000
247/247 [==============================] - 37s 150ms/step - loss: 0.0316 - mae: 0.1945 - mape: 7468.9976 - val_loss: 0.0561 - val_mae: 0.2824 - val_mape: 272.2151
Epoch 30/1000
247/247 [==============================] - 37s 150ms/step - loss: 0.0300 - mae: 0.1897 - mape: 19361.6074 - val_loss: 0.0565 - val_mae: 0.2817 - val_mape: 341.8980
Epoch 31/1000
247/247 [==============================] - 37s 149ms/step - loss: 0.0269 - mae: 0.1799 - mape: 11895.4844 - val_loss: 0.0537 - val_mae: 0.2715 - val_mape: 271.2910
Epoch 32/1000
247/247 [==============================] - 38s 151ms/step - loss: 0.0265 - mae: 0.1794 - mape: 11569.6650 - val

KeyboardInterrupt: ignored

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir "/content/drive/MyDrive/Computer Vision Final Project/model/logs"
